## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Aksarka,RU,66.5606,67.7975,66.15,28,61,18.32,broken clouds
1,1,Roald,NO,62.5833,6.1333,49.75,81,75,10.36,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,33.46,93,100,27.63,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,58.86,66,40,9.22,scattered clouds
4,4,Ponta Do Sol,PT,32.6667,-17.1000,66.83,68,57,3.87,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kapaa,US,22.0752,-159.3190,78.78,80,20,10.36,few clouds
8,8,Katsuura,JP,35.1333,140.3000,70.66,71,39,9.24,scattered clouds
11,11,Hilo,US,19.7297,-155.0900,75.27,87,100,6.91,overcast clouds
12,12,Road Town,VG,18.4167,-64.6167,78.89,79,100,17.76,overcast clouds
20,20,Samarai,PG,-10.6167,150.6667,80.71,65,86,11.86,overcast clouds
21,21,Rikitea,PF,-23.1203,-134.9692,76.86,73,11,11.41,few clouds
23,23,Atuona,PF,-9.8000,-139.0333,78.39,72,34,17.85,scattered clouds
24,24,Pemangkat,ID,1.1667,108.9667,86.18,67,76,11.27,broken clouds
26,26,Port Hedland,AU,-20.3167,118.5667,82.45,57,82,10.36,broken clouds
31,31,Santa Cruz Del Sur,CU,20.7147,-77.9956,76.01,84,100,6.08,overcast clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current Description    264
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Kapaa,US,22.0752,-159.3190,78.78,80,20,10.36,few clouds
8,8,Katsuura,JP,35.1333,140.3000,70.66,71,39,9.24,scattered clouds
11,11,Hilo,US,19.7297,-155.0900,75.27,87,100,6.91,overcast clouds
12,12,Road Town,VG,18.4167,-64.6167,78.89,79,100,17.76,overcast clouds
20,20,Samarai,PG,-10.6167,150.6667,80.71,65,86,11.86,overcast clouds


In [16]:
clean_travel_cities.columns

Index(['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Current Description'],
      dtype='object')

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kapaa,US,78.78,few clouds,22.0752,-159.3190,
8,Katsuura,JP,70.66,scattered clouds,35.1333,140.3000,
11,Hilo,US,75.27,overcast clouds,19.7297,-155.0900,
12,Road Town,VG,78.89,overcast clouds,18.4167,-64.6167,
20,Samarai,PG,80.71,overcast clouds,-10.6167,150.6667,
21,Rikitea,PF,76.86,few clouds,-23.1203,-134.9692,
23,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,
24,Pemangkat,ID,86.18,broken clouds,1.1667,108.9667,
26,Port Hedland,AU,82.45,broken clouds,-20.3167,118.5667,
31,Santa Cruz Del Sur,CU,76.01,overcast clouds,20.7147,-77.9956,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kapaa,US,78.78,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Katsuura,JP,70.66,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
11,Hilo,US,75.27,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Road Town,VG,78.89,overcast clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
20,Samarai,PG,80.71,overcast clouds,-10.6167,150.6667,Nuli Sapi
21,Rikitea,PF,76.86,few clouds,-23.1203,-134.9692,People ThankYou
23,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,Villa Enata
24,Pemangkat,ID,86.18,broken clouds,1.1667,108.9667,Ramah Kost KITA
26,Port Hedland,AU,82.45,broken clouds,-20.3167,118.5667,The Esplanade Hotel
31,Santa Cruz Del Sur,CU,76.01,overcast clouds,20.7147,-77.9956,


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kapaa,US,78.78,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
8,Katsuura,JP,70.66,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
11,Hilo,US,75.27,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Road Town,VG,78.89,overcast clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
20,Samarai,PG,80.71,overcast clouds,-10.6167,150.6667,Nuli Sapi
21,Rikitea,PF,76.86,few clouds,-23.1203,-134.9692,People ThankYou
23,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,Villa Enata
24,Pemangkat,ID,86.18,broken clouds,1.1667,108.9667,Ramah Kost KITA
26,Port Hedland,AU,82.45,broken clouds,-20.3167,118.5667,The Esplanade Hotel
37,Puerto Escondido,MX,80.87,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))